### Task One: Binary Representations
#### This task was all about working with bits. I had to implement four functions that manipulate bits in a 32-bit unsigned integer. These kinds of operations are essential in areas like cryptography and data compression, where you need to optimize data handling at the binary level.

### 1. **`rotl(x, n)`** – Rotates the bits of `x` to the left by `n` positions. Think of it like shifting bits around in a loop.

In [22]:
# Rotate Left Function
def rotl(x: int, n: int = 1) -> int:
    """
    Rotates the bits of a 32-bit unsigned integer to the left by n positions.
    """
    n = n % 32  # Keep n in the range 0-31
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

### Example Usage Of Rotl - This example rotates the bits of a number to the left by 4 positions and shows the result, including a case where all bits are set.

In [24]:
# Example usage of rotl
x = 0b00000000000000000000000000000001  # Binary representation of 1
rotated_value = rotl(x, 4)

# Output the original and rotated values
print(f"Original value (bin): {bin(x)}")
print(f"Rotated left by 4 (bin): {bin(rotated_value)}")

# Example with all bits set
x_all_set = 0xFFFFFFFF  # All 32 bits set to 1
rotated_all_set = rotl(x_all_set, 5)
print(f"Rotated left (all bits set): {bin(rotated_all_set)}") 


Original value (bin): 0b1
Rotated left by 4 (bin): 0b10000
Rotated left (all bits set): 0b11111111111111111111111111111111


### - Test Case For Rotate Left Function

In [26]:
import unittest

class TestRotateLeft(unittest.TestCase):

    def test_rotl_basic(self):
        self.assertEqual(rotl(0b0001, 4), 0b10000)

    def test_rotl_wraparound(self):
        self.assertEqual(rotl(0b10000000000000000000000000000000, 1), 
                         0b00000000000000000000000000000001)

    def test_rotl_zero_rotation(self):
        self.assertEqual(rotl(0b1010, 0), 0b1010)

unittest.main(argv=[''], verbosity=2, exit=False)


test_rotl_basic (__main__.TestRotateLeft.test_rotl_basic) ... ok
test_rotl_wraparound (__main__.TestRotateLeft.test_rotl_wraparound) ... ok
test_rotl_zero_rotation (__main__.TestRotateLeft.test_rotl_zero_rotation) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.025s

OK


### 2. **`rotr(x, n)`** – Same thing as `rotl`, but in the opposite direction (to the right).

In [27]:
# Rotate Right Function
def rotr(x: int, n: int = 1) -> int:
    """
    Rotates the bits of a 32-bit unsigned integer to the right by n positions.
    """
    n = n % 32  # Keep n in the range 0-31
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

### Example Usage Of Rotr - This example demonstrates rotating a number's bits to the right by 1 and 2 positions, including a case with a repeating bit pattern.

In [28]:
# Example usage of rotr
x = 0b10000000000000000000000000000000  # Binary representation with only the most significant bit set
rotated_value = rotr(x, 1)

# Output the original and rotated values
print(f"Original value (bin): {bin(x)}")
print(f"Rotated right by 1 (bin): {bin(rotated_value)}")

# Example with a pattern
x_pattern = 0b11001100110011001100110011001100
rotated_pattern = rotr(x_pattern, 2)
print(f"Original value (bin): {bin(x_pattern)}") 
print(f"Rotated right by 2 (bin): {bin(rotated_pattern)}")


Original value (bin): 0b10000000000000000000000000000000
Rotated right by 1 (bin): 0b1000000000000000000000000000000
Original value (bin): 0b11001100110011001100110011001100
Rotated right by 2 (bin): 0b110011001100110011001100110011


### - Test Case For Rotate Right Function

In [33]:
import unittest

class TestRotateRight(unittest.TestCase):

    def test_rotr_basic(self):
        """Test basic right rotation by 1 position."""
        self.assertEqual(rotr(0b1000, 1), 0b0100)

    def test_rotr_wraparound(self):
        """Test wrap-around where the least significant bit moves to the most significant bit."""
        self.assertEqual(rotr(0b00000000000000000000000000000001, 1), 
                         0b10000000000000000000000000000000)

    def test_rotr_zero_rotation(self):
        """Test rotation by 0 positions (should return the original value)."""
        self.assertEqual(rotr(0b1010, 0), 0b1010)

unittest.main(argv=[''], verbosity=2, exit=False)


test_rotl_basic (__main__.TestRotateLeft.test_rotl_basic) ... ok
test_rotl_wraparound (__main__.TestRotateLeft.test_rotl_wraparound) ... ok
test_rotl_zero_rotation (__main__.TestRotateLeft.test_rotl_zero_rotation) ... ok
test_rotr_basic (__main__.TestRotateRight.test_rotr_basic)
Test basic right rotation by 1 position. ... ok
test_rotr_wraparound (__main__.TestRotateRight.test_rotr_wraparound)
Test wrap-around where the least significant bit moves to the most significant bit. ... ok
test_rotr_zero_rotation (__main__.TestRotateRight.test_rotr_zero_rotation)
Test rotation by 0 positions (should return the original value). ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK


### 3. **`ch(x, y, z)`** – This function picks bits from either `y` or `z`, depending on the value of `x`. It's a conditional bit selector.

In [29]:
# Choose Function 
def ch(x: int, y: int, z: int) -> int:
    """
    Chooses bits from y where x has bits set to 1, and from z where x has bits set to 0.
    """
    return (x & y) | (~x & z) & 0xFFFFFFFF

### Example Usage Of CH - This example demonstrates the ch function, which selects bits from y where x has 1s and from z where x has 0s, producing a result based on the selector bits in x.

In [30]:
# Example usage of ch
x = 0b10101010  # Selector bits
y = 0b11111111  # Bits to choose when x has 1s
z = 0b00000000  # Bits to choose when x has 0s

chosen_bits = ch(x, y, z)

# Output the result of the choose function
print(f"Selector (x):   {bin(x)}") 
print(f"Option 1 (y):   {bin(y)}")
print(f"Option 2 (z):   {bin(z)}")
print(f"Chosen result:  {bin(chosen_bits)}")

Selector (x):   0b10101010
Option 1 (y):   0b11111111
Option 2 (z):   0b0
Chosen result:  0b10101010


##### 4. **`maj(x, y, z)`** – Calculates the majority bit for each position across `x`, `y`, and `z`. If at least two of the inputs have a `1` in a position, the result gets a `1`.

In [31]:
# Majority Function
def maj(x: int, y: int, z: int) -> int:
    """
    Computes the majority function on the bits of x, y, and z.
    """
    return (x & y) | (x & z) | (y & z) & 0xFFFFFFFF

In [32]:
# Example usage of maj
x = 0b10101010  # Input 1
y = 0b11110000  # Input 2
z = 0b00001111  # Input 3

majority_bits = maj(x, y, z)

# Output the result of the majority function
print(f"Input 1 (x):    {bin(x)}") 
print(f"Input 2 (y):    {bin(y)}")
print(f"Input 3 (z):    {bin(z)}")
print(f"Majority result: {bin(majority_bits)}")


Input 1 (x):    0b10101010
Input 2 (y):    0b11110000
Input 3 (z):    0b1111
Majority result: 0b10101010
